In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
import numpy as np

In [7]:
sales_data = pd.read_csv("train_v9rqX0R.csv")
validation_data = pd.read_csv("test_AbJTz2l.csv")

In [8]:
sales_data = sales_data[["Item_MRP","Outlet_Type", "Outlet_Identifier", "Item_Outlet_Sales", "Outlet_Location_Type"]]
categorical_cols = sales_data.select_dtypes(include=['object', 'category']).columns.tolist()
print("Categorical Columns:", categorical_cols)

Categorical Columns: ['Outlet_Type', 'Outlet_Identifier', 'Outlet_Location_Type']


In [9]:
X_train = sales_data.drop(['Item_Outlet_Sales'], axis=1)
y_train = sales_data['Item_Outlet_Sales']

In [10]:
# initialize the model
cat_model = CatBoostRegressor(iterations=1000, learning_rate=0.01, depth=6,\
                          loss_function='RMSE' ,cat_features=list(categorical_cols),nan_mode='Min')

cat_model.fit(
    X_train, 
    y_train.values.ravel(),
    logging_level='Silent',
)



In [11]:
# TESTING ON VALIDATION DATA

In [12]:
id_column = validation_data[['Item_Identifier', 'Outlet_Identifier']]
validation_data = validation_data[["Item_MRP","Outlet_Type", "Outlet_Identifier", "Outlet_Location_Type"]]



In [13]:
y_pred = cat_model.predict(validation_data)

In [14]:
id_column_reset = id_column.reset_index(drop=True)

predictions = pd.DataFrame({
    'Item_Identifier': id_column_reset['Item_Identifier'],
    'Outlet_Identifier': id_column_reset['Outlet_Identifier'],
    'Predicted_Sales': y_pred
})

print(predictions.head())
predictions.columns = ["Item_Identifier","Outlet_Identifier", "Item_Outlet_Sales" ]
predictions.to_csv("prediction_catboost.csv")

  Item_Identifier Outlet_Identifier  Predicted_Sales
0           FDW58            OUT049      1752.189168
1           FDW14            OUT017      1416.520863
2           NCN55            OUT010       591.527035
3           FDQ58            OUT017      2554.324758
4           FDY38            OUT027      6114.609366


In [15]:
# RANK 171